In [ ]:
#跑起来太慢了，有点浪费服务器资源和网络。弃用
#看第二版

import json
import pandas as pd
from tqsdk import TqApi, TqAuth, TqBacktest, TargetPosTask, TqSim
from datetime import date
from tqsdk.tafunc import ma,ema
from tqsdk.exceptions import BacktestFinished
import math

def create_backtest_record(R):
  data = {
      "策略-参数1":R["Strategy_name"],
      "策略-参数2":R["param_2"],
      "策略-参数3":R["param_3"],
      "标的":R["标的"],
      "开始时间":R["start_date"],
      "结束时间":R["end_date"],

      "保证金" : R["保证金"],
      "使用金额" : R["保证金"]*10,

      "起始金额" : R["report"]["start_balance"],
      "结束金额" : R["report"]["end_balance"],

      "最大回撤" : R["report"]["max_drawdown"]*R["report"]["start_balance"]/(R["保证金"]*10),
      "盈亏比" : R["report"]["profit_loss_ratio"],
      "胜率" : R["report"]["winning_rate"],

      "收益率" : (R["report"]["end_balance"]-R["report"]["start_balance"])/max(R["保证金"]*10,R["report"]["max_drawdown"]*R["report"]["start_balance"]*2)*100

  }

  df = pd.DataFrame(data, index=[0])
  return df

#计算账户内现金能买多少手合约
def calculate_volume(acc, symbol):
    volume = (acc.get_account().available)/(acc.get_margin(symbol))
    return math.floor(volume)

def calculate_vwma(data, window_size):
    """
    计算成交量加权移动平均线（VWMA）
    :param data: 包含日期、收盘价和成交量的数据框
    :param window_size: 计算VWMA的窗口大小
    :return: 包含原始数据和VWMA的数据框
    """
    # 确保数据框中的价格和成交量列是数值类型
    data = data.copy()
    data['close'] = pd.to_numeric(data['close'])
    data['volume'] = pd.to_numeric(data['volume'])
    # 计算价格与成交量的乘积
    data['价格_成交量乘积'] = data['close'] * data['volume']
    # 计算VWMA
    data['VWMA'] = data['价格_成交量乘积'].rolling(window=window_size).sum() / data['volume'].rolling(window=window_size).sum()
    return data["VWMA"]



class Strategy_XMA:
    def __init__(self,user,short,long,X_MA,k=0.5):
        self.user = user
        self.short = short
        self.long = long
        self.X_MA = X_MA
        self.k = k
        self.result_dict = {"Strategy_name":self.X_MA,
                            "param_2" : str(self.short),
                            "param_3" : str(self.long),
                            "标的":None,
                            "保证金" : None,
                            "start_date":None,"end_date":None,"report":None}
        
        
    def run_strategy(self,symbol,start_date,end_date):
        self.result_dict["标的"] = symbol
        self.result_dict["start_date"] = start_date
        self.result_dict["end_date"] = end_date
        
        #时间转换
        y,m,d = start_date.split("-")
        y,m,d = int(y),int(m),int(d)
        e_y,e_m,e_d = end_date.split("-")
        e_y,e_m,e_d = int(e_y),int(e_m),int(e_d)

        acc = TqSim(1000000)
        api = TqApi(acc,backtest=TqBacktest(start_dt=date(y,m,d), end_dt=date(e_y,e_m,e_d)),web_gui=True,auth=TqAuth(self.user["username"], self.user["password"]))
        q = api.get_quote(symbol)
        self.result_dict["保证金"] = q.margin
        print("策略开始运行{}-{}-{}-{}".format(self.X_MA,self.short,self.long,symbol))
        data_length =self.long + 3  # k线数据长度
        # "duration_seconds=60"为一分钟线, 日线的duration_seconds参数为: 24*60*60
        klines = api.get_kline_serial(symbol, duration_seconds=60*60*24, data_length=data_length)
        target_pos = TargetPosTask(api, symbol)
        
        try:
            while True:
                api.wait_update()
                A=0

                if api.is_changing(klines.iloc[-1], "datetime"):  # 产生新k线:重新计算SMA
                    A=1
                    if self.X_MA == "MA":

                        short_avg = ma(klines["close"], self.short)  # 短周期
                        long_avg = ma(klines["close"],  self.long)  # 长周期

                    elif self.X_MA == "EMA":
                        short_avg = ema(klines["close"], self.short)  # 短周期
                        long_avg = ema(klines["close"],  self.long)  # 长周期

                    elif self.X_MA == "VWMA":
                        short_avg = calculate_vwma(klines, self.short)  # 短周期
                        long_avg = calculate_vwma(klines,  self.long)  # 长周期


                    elif self.X_MA == "MA+kσ":
                        short_avg = ma(klines["close"], self.short)  # 短周期
                        long_avg_top= ma(klines["close"],  self.long) + klines["close"].rolling(window=self.long).std()*0.2  # 长周期上轨道
                        long_avg_bottom = ma(klines["close"],  self.long,) -  klines["close"].rolling(window=self.long).std()*0.2 # 长周期下轨道

                        # 均线下穿，做空
                        if long_avg_bottom.iloc[-3] < short_avg.iloc[-3] and long_avg_bottom.iloc[-2] > short_avg.iloc[-2]:
                            target_pos.set_target_volume(-10)

                        # 均线上穿，做多
                        if short_avg.iloc[-3] < long_avg_top.iloc[-3] and short_avg.iloc[-2] > long_avg_top.iloc[-2]:
                            target_pos.set_target_volume(10)
                        continue

                    else:
                        print("X_MA参数输入错误")
                        break
                if A==1:
                    # 上穿，做多
                    if long_avg.iloc[-3] > short_avg.iloc[-3] and long_avg.iloc[-2] < short_avg.iloc[-2]:
                        target_pos.set_target_volume(10)


                    # 下穿，做空
                    if short_avg.iloc[-3] > long_avg.iloc[-3] and short_avg.iloc[-2] < long_avg.iloc[-2]:
                        target_pos.set_target_volume(-10)

                    
        except BacktestFinished: 
            print("回测结束")
            # 回测结束时会执行这里的代码
            self.result_dict["report"] = acc.tqsdk_stat
            api.close()

            return self.result_dict
        
#读取标的池和账户信息
with open('c:/Users/zhaoc/OneDrive/LH2024/HL2024-flask/策略研究/账户.json','r',encoding='utf8')as fp:
    json_data = json.load(fp)
    user = json_data['TQ_acc']

with open('c:/Users/zhaoc/OneDrive/LH2024/HL2024-flask/策略研究/标的池/20250319标的池.json','r',encoding='utf8')as fp:
    json_data = json.load(fp)
    symbols = list(json_data['回测池——20250320'].values())

with open('c:/Users/zhaoc/OneDrive/LH2024/HL2024-flask/策略研究/策略池/策略池.json','r',encoding='utf8')as fp:
    json_data = json.load(fp)
    strategy_list = json_data['双均线策略-均线类型']
    param_list = json_data['双均线策略-均线周期']


# #验证标的池合约有效性
# symbol_fail_list = []
# acc = TqSim()
# for symbol in symbols[15:]:
#     y,m,d = symbol["start_date"].split("-")
#     y,m,d = int(y),int(m),int(d)

#     e_y,e_m,e_d = symbol["end_date"].split("-")
#     e_y,e_m,e_d = int(e_y),int(e_m),int(e_d)
#     api = TqApi(acc,backtest=TqBacktest(start_dt=date(y,m,d), end_dt=date(e_y,e_m,e_d),),auth=TqAuth(user["username"], user["password"]))
#     try:
#         q = api.get_quote(symbol["symbol"])
#         print("{} 合约有效".format(symbol))
#         print(q.last_price)
#     except:
#         print("{} 合约无效".format(symbol))
#         symbol_fail_list.append(symbol)
#         symbols.remove(symbol)

#     api.close()

# print("无效合约列表：",symbol_fail_list)

在使用天勤量化之前，默认您已经知晓并同意以下免责条款，如果不同意请立即停止使用：https://www.shinnytech.com/blog/disclaimer/


In [ ]:
#单个策略运行时间大约在3分中左右，全部的玉米跑完：需要3*16*8=384分钟，需要6.4个小时
#验证策略有效性
backtest_record_list = []
for xma in list(strategy_list.values())[:]:
    for s,l in list(param_list.values())[:]:
        print(xma,s,l,end=" ")

        for symbol in symbols[:8]:
            print(symbol)
            A = Strategy_XMA(user,short=s,long=l,X_MA=xma)
            symbol,s_date,e_date = symbol["symbol"],symbol["start_date"],symbol["end_date"]
            R = A.run_strategy(symbol,s_date,e_date)
            backtest_record_list.append(create_backtest_record(R))

backtest_record_df = pd.concat(backtest_record_list)
backtest_record_df.to_csv("策略池_标的池.csv",index=False)


MA 5 15 {'symbol': 'DCE.c2505', 'start_date': '2024-01-15', 'end_date': '2025-01-15'}
    INFO - 您可以访问 http://127.0.0.1:65141 查看策略绘制出的 K 线图形。
策略开始运行MA-5-15
    INFO - 模拟交易下单 TQSIM, PYSDK_target_4e405ef456c189edb5cdb189f8ceec80: 时间: 2024-06-17 21:00:00.000000, 合约: DCE.c2505, 开平: OPEN, 方向: BUY, 手数: 10, 价格: 2455.0
    INFO - 模拟交易委托单 TQSIM, PYSDK_target_4e405ef456c189edb5cdb189f8ceec80: 全部成交
    INFO - 模拟交易下单 TQSIM, PYSDK_target_90e0ff91ae76dcc8e11b5049f9549977: 时间: 2024-06-19 21:00:00.000000, 合约: DCE.c2505, 开平: CLOSE, 方向: SELL, 手数: 10, 价格: 2442.0
    INFO - 模拟交易委托单 TQSIM, PYSDK_target_90e0ff91ae76dcc8e11b5049f9549977: 全部成交
    INFO - 模拟交易下单 TQSIM, PYSDK_target_16351a4b45e117c2e9501cd20a5bbe2e: 时间: 2024-06-19 21:00:00.000000, 合约: DCE.c2505, 开平: OPEN, 方向: SELL, 手数: 10, 价格: 2442.0
    INFO - 模拟交易委托单 TQSIM, PYSDK_target_16351a4b45e117c2e9501cd20a5bbe2e: 全部成交
    INFO - 模拟交易下单 TQSIM, PYSDK_target_e01a1692793926c1e06572baf8371f7e: 时间: 2024-06-27 21:00:00.000000, 合约: DCE.c2505, 开平: CLOSE, 方向: BU

KeyboardInterrupt: 

In [ ]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

def run_backtest(xma, s, l, symbol):
    symbol, s_date, e_date = symbol["symbol"], symbol["start_date"], symbol["end_date"]
    A = Strategy_XMA(user, short=s, long=l, X_MA=xma)
    R = A.run_strategy(symbol, s_date, e_date)
    return create_backtest_record(R)

def main():
    backtest_record_list = []
    with ThreadPoolExecutor() as executor:
        # 使用列表推导式创建一个任务列表
        tasks = [executor.submit(run_backtest, xma, s, l, symbol)
                 for xma in list(strategy_list.values())
                 for s, l in list(param_list.values())
                 for symbol in symbols[:8]]

        # 使用tqdm来跟踪任务进度
        for future in tqdm(as_completed(tasks), total=len(tasks)):
            backtest_record_list.append(future.result())

    # 将结果合并为DataFrame并保存为CSV文件
    backtest_record_df = pd.concat(backtest_record_list)
    backtest_record_df.to_csv("策略池_标的池.csv", index=False)

if __name__ == "__main__":
    main()

  0%|          | 0/128 [00:11<?, ?it/s]
Task was destroyed but it is pending!
task: <Task pending name='Task-7273582' coro=<TqApi._notify_watcher() running at c:\Users\zhaoc\.conda\envs\HL2025\lib\site-packages\tqsdk\api.py:3795> cb=[TqBaseApi._on_task_done()]>
Task was destroyed but it is pending!
task: <Task pending name='Task-7273581' coro=<TqBaseApi._windows_patch() running at c:\Users\zhaoc\.conda\envs\HL2025\lib\site-packages\tqsdk\baseApi.py:143> cb=[TqBaseApi._on_task_done()]>
Task was destroyed but it is pending!
task: <Task pending name='Task-7368540' coro=<TqBaseApi._windows_patch() running at c:\Users\zhaoc\.conda\envs\HL2025\lib\site-packages\tqsdk\baseApi.py:143> cb=[TqBaseApi._on_task_done()]>
Task was destroyed but it is pending!
task: <Task pending name='Task-7368541' coro=<TqApi._notify_watcher() running at c:\Users\zhaoc\.conda\envs\HL2025\lib\site-packages\tqsdk\api.py:3795> cb=[TqBaseApi._on_task_done()]>
c:\Users\zhaoc\.conda\envs\HL2025\lib\json\decoder.py:353: R

    INFO - 您可以访问 http://127.0.0.1:64151 查看策略绘制出的 K 线图形。


Task was destroyed but it is pending!
task: <Task pending name='Task-11183055' coro=<TqBaseApi._windows_patch() running at c:\Users\zhaoc\.conda\envs\HL2025\lib\site-packages\tqsdk\baseApi.py:143> cb=[TqBaseApi._on_task_done()]>
Task was destroyed but it is pending!
task: <Task pending name='Task-11183089' coro=<TqBaseApi._windows_patch() running at c:\Users\zhaoc\.conda\envs\HL2025\lib\site-packages\tqsdk\baseApi.py:143> cb=[TqBaseApi._on_task_done()]>
Task was destroyed but it is pending!
task: <Task pending name='Task-11183090' coro=<TqApi._notify_watcher() running at c:\Users\zhaoc\.conda\envs\HL2025\lib\site-packages\tqsdk\api.py:3795> cb=[TqBaseApi._on_task_done()]>
Task was destroyed but it is pending!
task: <Task pending name='Task-11183162' coro=<TqApi._notify_watcher() running at c:\Users\zhaoc\.conda\envs\HL2025\lib\site-packages\tqsdk\api.py:3795> cb=[TqBaseApi._on_task_done()]>
Task was destroyed but it is pending!
task: <Task pending name='Task-11183161' coro=<TqBaseApi._

    INFO - 您可以访问 http://127.0.0.1:64154 查看策略绘制出的 K 线图形。


Task was destroyed but it is pending!
task: <Task pending name='Task-11183329' coro=<TqApi._notify_watcher() running at c:\Users\zhaoc\.conda\envs\HL2025\lib\site-packages\tqsdk\api.py:3795> cb=[TqBaseApi._on_task_done()]>
Task was destroyed but it is pending!
task: <Task pending name='Task-11183328' coro=<TqBaseApi._windows_patch() running at c:\Users\zhaoc\.conda\envs\HL2025\lib\site-packages\tqsdk\baseApi.py:143> cb=[TqBaseApi._on_task_done()]>
Task was destroyed but it is pending!
task: <Task pending name='Task-11183361' coro=<TqBaseApi._windows_patch() running at c:\Users\zhaoc\.conda\envs\HL2025\lib\site-packages\tqsdk\baseApi.py:143> cb=[TqBaseApi._on_task_done()]>
Task was destroyed but it is pending!
task: <Task pending name='Task-11183362' coro=<TqApi._notify_watcher() running at c:\Users\zhaoc\.conda\envs\HL2025\lib\site-packages\tqsdk\api.py:3795> cb=[TqBaseApi._on_task_done()]>
Task was destroyed but it is pending!
task: <Task pending name='Task-11183395' coro=<TqBaseApi._

...

    INFO - 您可以访问 http://127.0.0.1:64181 查看策略绘制出的 K 线图形。
    INFO - 您可以访问 http://127.0.0.1:64184 查看策略绘制出的 K 线图形。


Task was destroyed but it is pending!
task: <Task pending name='Task-11137638' coro=<TqApi._notify_watcher() running at c:\Users\zhaoc\.conda\envs\HL2025\lib\site-packages\tqsdk\api.py:3795> cb=[TqBaseApi._on_task_done()]>
Task was destroyed but it is pending!
task: <Task pending name='Task-11137637' coro=<TqBaseApi._windows_patch() running at c:\Users\zhaoc\.conda\envs\HL2025\lib\site-packages\tqsdk\baseApi.py:143> cb=[TqBaseApi._on_task_done()]>
Task was destroyed but it is pending!
task: <Task pending name='Task-11151870' coro=<TqApi._notify_watcher() running at c:\Users\zhaoc\.conda\envs\HL2025\lib\site-packages\tqsdk\api.py:3795> cb=[TqBaseApi._on_task_done()]>
Task was destroyed but it is pending!
task: <Task pending name='Task-11151869' coro=<TqBaseApi._windows_patch() running at c:\Users\zhaoc\.conda\envs\HL2025\lib\site-packages\tqsdk\baseApi.py:143> cb=[TqBaseApi._on_task_done()]>
Task was destroyed but it is pending!
task: <Task pending name='Task-11167046' coro=<TqBaseApi._

    INFO - 您可以访问 http://127.0.0.1:64193 查看策略绘制出的 K 线图形。


Task was destroyed but it is pending!
task: <Task pending name='Task-11183523' coro=<TqBaseApi._windows_patch() running at c:\Users\zhaoc\.conda\envs\HL2025\lib\site-packages\tqsdk\baseApi.py:143> cb=[TqBaseApi._on_task_done()]>
Task was destroyed but it is pending!
task: <Task pending name='Task-11183524' coro=<TqApi._notify_watcher() running at c:\Users\zhaoc\.conda\envs\HL2025\lib\site-packages\tqsdk\api.py:3795> cb=[TqBaseApi._on_task_done()]>
c:\Users\zhaoc\.conda\envs\HL2025\lib\json\decoder.py:353: RuntimeWarning: coroutine 'TqApi._notify_watcher' was never awaited
  obj, end = self.scan_once(s, idx)
c:\Users\zhaoc\.conda\envs\HL2025\lib\json\decoder.py:353: RuntimeWarning: coroutine 'TqBaseApi._windows_patch' was never awaited
  obj, end = self.scan_once(s, idx)
c:\Users\zhaoc\.conda\envs\HL2025\lib\json\decoder.py:353: RuntimeWarning: coroutine 'TqApi._notify_watcher' was never awaited
  obj, end = self.scan_once(s, idx)


    INFO - 您可以访问 http://127.0.0.1:64208 查看策略绘制出的 K 线图形。
    INFO - 您可以访问 http://127.0.0.1:64211 查看策略绘制出的 K 线图形。
    INFO - 您可以访问 http://127.0.0.1:64216 查看策略绘制出的 K 线图形。
    INFO - 您可以访问 http://127.0.0.1:64223 查看策略绘制出的 K 线图形。
    INFO - 您可以访问 http://127.0.0.1:64235 查看策略绘制出的 K 线图形。
    INFO - 您可以访问 http://127.0.0.1:64238 查看策略绘制出的 K 线图形。
    INFO - 您可以访问 http://127.0.0.1:64242 查看策略绘制出的 K 线图形。
    INFO - 您可以访问 http://127.0.0.1:64248 查看策略绘制出的 K 线图形。    INFO - 您可以访问 http://127.0.0.1:64247 查看策略绘制出的 K 线图形。

    INFO - 您可以访问 http://127.0.0.1:64251 查看策略绘制出的 K 线图形。
策略开始运行EMA-5-15
策略开始运行EMA-5-15
策略开始运行EMA-5-15
    INFO - 您可以访问 http://127.0.0.1:64264 查看策略绘制出的 K 线图形。
    INFO - 您可以访问 http://127.0.0.1:64267 查看策略绘制出的 K 线图形。
策略开始运行EMA-5-15
    INFO - 您可以访问 http://127.0.0.1:64279 查看策略绘制出的 K 线图形。
    INFO - 您可以访问 http://127.0.0.1:64281 查看策略绘制出的 K 线图形。
    INFO - 您可以访问 http://127.0.0.1:64283 查看策略绘制出的 K 线图形。
策略开始运行EMA-10-30
策略开始运行EMA-5-15
策略开始运行EMA-10-30
    INFO - 您可以访问 http://127.0.0.1:64308 查看策略绘制出的 K 线图